In [0]:
!pip -q install opencc-python-reimplemented
from google.colab import drive
drive.mount('/gdrive')

In [0]:
from collections import Counter
from opencc import OpenCC
import progressbar
import requests
import multiprocessing
import time
import re

In [0]:
header={
"Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
"Accept-encoding": "gzip, deflate, br",
"Accept-language": "en,zh-CN;q=0.9,zh;q=0.8",
"User-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.37 (KHTML, like Gecko) Chrome/81.0.4017.1 Safari/537.37 Edg/81.0.389.3"
}

In [0]:
cid_pattern = re.compile(r'"cid":[\d]*')
bvid_pattern = re.compile(r'"bvid":"BV[\w]*"')
def fetch_with_bv(bv):
    url = "https://www.bilibili.com/video/"+bv
    page = requests.get(url, headers = header)
    page.encoding = "utf-8"
    page = page.text
    
    cids = cid_pattern.findall(page)
    cids = [cid[6:] for cid in cids]

    BVs = bvid_pattern.findall(page)
    BVs = [bv[8:-1] for bv in BVs]

    return cids, BVs

In [0]:
comment_pattern = re.compile(r'">[\S]*</d>')
def get_comment_with_cid(cid):
    url = "https://comment.bilibili.com/"+str(cid)+".xml"
    page = requests.get(url, headers = header)
    page.encoding = "utf-8"
    page = page.text
    #comments = comment_pattern.findall(page)
    comments = comment_pattern.findall(page)
    comments = [comment[2:-4] for comment in comments]
    return comments

In [0]:
cids, BVs = fetch_with_bv("av314")
total_comments = []

for cid in cids:
    comments = get_comment_with_cid(cid)
    if(len(comments) < 5):
        print(cid)
    total_comments.extend(comments)
print(total_comments)

In [0]:
def test_if_ip_blocked(bv):
    url = "https://www.bilibili.com/video/"+bv
    page = requests.get(url, headers = header)
    page.encoding = "utf-8"
    page = page.text

    warn_text = re.search("该次访问请求被拒绝", page)

    return warn_text != None

In [0]:
visited_BVs = set()
bv_list = ["BV1u4411L7ha","av314","av170001","BV1ns411B7J2","BV1Qs411X7QR","BV1zs411S7sz"]
cid_list = []

In [0]:
print_every = 50
flag = False

i=0
while(len(bv_list)>0):
    i+=1
    bv = bv_list[0]
    bv_list.pop(0)
    visited_BVs.add(bv)
    cids, BVs = fetch_with_bv(bv)

    if(len(cids) == 0 and test_if_ip_blocked(bv) == True):
        print("trigger anti-spyider\n quiting...")
        break

    for cid, bv in zip(cids,BVs):
        if bv not in visited_BVs:
            bv_list.append(bv)
            cid_list.append(cid)
    
    if(i%print_every == 0):
        print(i)

    if(i > 10000):
        flag = True
        print("reach max pages available")
        break

if flag == False:
    print("Complete search with "+str(i)+" pages")
else:
    print("incomplete search with "+str(i)+" pages")

In [0]:
print(len(cid_list))

In [0]:
with open("cids.txt","a") as fout:
    fout.write("\n".join(cid_list))

In [0]:
with open("cids.txt","r") as fin:
    cids_str = fin.read()

print(cids_str[:100])

In [0]:
!cp "cids.txt" "/gdrive/My Drive/bili-filter/cids.txt"

In [0]:
with open("cids.txt","r",encoding='utf-8') as fin:
    cid_list = fin.read().split()

In [0]:
print(len(total_comments))

In [0]:
comments = []
print(len(cid_list))
pool = multiprocessing.Pool(processes = 8)

async_res_list = []

for cid in cid_list:
    async_res = pool.apply_async( get_comment_with_cid, (cid,))
    async_res_list.append( async_res )

for async_res in progressbar.progressbar(async_res_list):
    comments.extend(async_res.get())

print(len(comments))
print(comments[-100:])
pool.close()
pool.join() 

In [0]:
#filter for chinese japanese number and english
filter_for_words = re.compile('[^\u4E00-\u9FA5\u0030-\u0039\u0041-\u005a\u0061-\u007a\u3040-\u31FF]')
#convert traditional chinese to simplified chinese
cc = OpenCC('t2s')

preprocessed_comments = []
sum1=0
sum2=0
for comment in comments:
    temp = cc.convert(filter_for_words.sub('',comment))
    if(len(temp)>1):
        sum1+=1
        preprocessed_comments.append(temp)
    else:
        sum2+=1

In [0]:
print(sum1,sum2,len(comments))

In [0]:
print(len(preprocessed_comments))

In [0]:
comments_len = Counter([len(x) for x in preprocessed_comments])
print(comments_len)

In [0]:
with open("comments.txt","w") as fout:
    fout.write("\n".join(preprocessed_comments))
!cp "comments.txt" "/gdrive/My Drive/bili-filter/comments.txt"

In [0]:
print(get_comment_with_cid(cid_list[0]))

In [0]:
!ls -lh "/gdrive/My Drive/bili-filter/comments.txt"

In [0]:
print(preprocessed_comments[123:134])

In [0]:
print(comments[:10])
filter_for_words.sub('',comments[7])